In [1]:
import pandas as pd

In [3]:
bigrams = pd.read_csv("bigram_data.csv")

In [5]:
bigrams.head()

,custom_id,twitter_handle,bigram
0,263,RepFinkenauer,"(health,care)"
1,263,RepFinkenauer,"(care,act)"
2,263,RepFinkenauer,"(million,americans)"
3,263,RepFinkenauer,"(pre,existing)"
4,263,RepFinkenauer,"(existing,conditions)"


In [51]:
top = bigrams["bigram"].value_counts().iloc[:10]
top.head()

Index(['(health,care)', '(american,people)', '(president,trump)',
       '(look,forward)', '(climate,change)', '(town,hall)', '(mueller,report)',
       '(looking,forward)', '(trump,administration)', '(pre,existing)'],
      dtype='object')

In [67]:
def separate_by_id(bigrams_df):
    df_l = []
    for cid in bigrams["custom_id"].unique():
        df = bigrams[bigrams["custom_id"]==cid]
        df_l.append(df)
    return df_l

In [68]:
df_l = separate_by_id(bigrams)

In [69]:
def bigram_freq(df_list: [pd.DataFrame], bigrams: [str]):
    dict_list = []
    for df in df_list:
        v = df["bigram"].value_counts()
        d = {"custom_id": df["custom_id"].iloc[0]}
        for bigram in bigrams:
            d[bigram] = 0
            if bigram in v:
                d[bigram] = v[bigram]
        dict_list.append(d)
    return dict_list

In [70]:
dl = bigram_freq(df_l, top.index)

In [71]:
bigram_count = pd.DataFrame(dl).set_index("custom_id")
bigram_count.head()

,"(american,people)","(climate,change)","(health,care)","(look,forward)","(looking,forward)","(mueller,report)","(pre,existing)","(president,trump)","(town,hall)","(trump,administration)"
custom_id,,,,,,,,,,
263,0,1,5,1,2,0,3,1,1,0
516,0,1,0,6,1,0,0,0,8,0
242,3,0,0,2,0,0,0,3,0,0
142,12,4,3,0,1,18,2,1,0,4
528,2,4,7,3,0,0,1,4,1,4
